In [5]:
import spacy
import re

In [6]:
from pandas import DataFrame

In [7]:
import string

In [8]:
import pandas as pd

In [9]:
from numpy import dot
from numpy.linalg import norm

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [13]:
#import fasttext
from sentence_transformers import SentenceTransformer, util

In [14]:
nlp = spacy.load('en_core_web_lg')

In [15]:
model = SentenceTransformer('stsb-roberta-large')

In [17]:
def computeSBertSimilarity(emb1, emb2):
    #emb1 = model.encode(sentence1)
    #emb2 = model.encode(sentence2)
    
    cos_sim = util.pytorch_cos_sim(emb1, emb2)
    #print("Cosine-Similarity:", cos_sim)
    return cos_sim

In [19]:
def getPercentageOfPredicted(a):
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    counter = 0
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))

In [20]:
def getNumberOfOracled(a):
    counter = 0
    
    for x in a:
        if(x[4] != "predicted"):
            counter = counter + 1
            
    return counter

In [21]:
def getNumberOfPredicted(a):
    counter = 0
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter

In [22]:
def getPercentageCorrectPredictions(a):
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    counter = counter + len(unclassifiedSamples4GridSearch)
            
    return counter / (len(unclassifiedSamples4GridSearch) + len(classifiedSamples4GridSearch))

def getLocalAutomationRate(a):
    counter = 0
    
    global unclassifiedSamples4GridSearch, classifiedSamples4GridSearch
    
    for x in a:
        if(x[4] == "predicted"):
            counter = counter + 1
            
    return counter / (len(a))

def getLocalAccuracy(a):
    
    counter = 0
    for x in a:
        if(x[1] == x[3]):
            counter = counter + 1

    
            
    return counter / (len(classifiedSamples4GridSearch))

# If beta = 1, then accuracy and automationRate have same importance
# If beta < 1, then accuracy is more important than automation rate
# If beta > 1, then automation rate is more important than accuracy
def getHarmonicScore (accuracy, automationRate, beta):
    
    if(accuracy == 0 and automationRate == 0):
        return 0
    
    score = (1 + beta ** 2) * (accuracy * automationRate) / ((accuracy * (beta ** 2)) + automationRate)
    #s1 = (1 + beta**2)
    #s2 = (accuracy * automationRate)
    #s3 = (accuracy*(beta**2)) + automationRate
    #print(s1)
    #print(s2)
    #print(s3)
    
    return score

In [23]:
#ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted
def getLabelDistribution(a):
    
    predictedDict = {}
    actualDict = {}

    for label in uniqueLabels:
        predictedDict.update({label: 0})
        
    for label in uniqueLabels:
        actualDict.update({label: 0})
        
    print(predictedDict)
    print(actualDict)
    
    for x in a:
        predictedLabel = x[1]
        actualLabel = x[3]
        
        numOfPredicted = predictedDict[predictedLabel]
        numOfActual = actualDict[actualLabel]
        
        numOfPredicted = numOfPredicted + 1
        numOfActual = numOfActual + 1 
        
        predictedDict.update({predictedLabel: numOfPredicted})
        actualDict.update({actualLabel: numOfActual})
        
    print("prediction dict: ", predictedDict)
    print("actual dict: ", actualDict)

In [24]:
def cosineSimilarity(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

In [28]:
'''
 STRUCTURE OF allSamplesWithLabelThird: ticketText, ticketLabel, TF-IDF, wordVector

 STRUCTURE OF classifiedSamplesInit: ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
'''

'\n STRUCTURE OF allSamplesWithLabelThird: ticketText, ticketLabel, TF-IDF, wordVector\n\n STRUCTURE OF classifiedSamplesInit: ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector\n'

In [29]:
def computeCosineSimilarity(sentence1, sentence2):
    doc1 = nlp(sentence1)
    doc2 = nlp(sentence2)
    
    similarity = doc1.similarity(doc2)
    #print(similarity)
    return similarity

In [30]:
def preprocessing(sentence):
    
    doc = nlp(sentence)
    inputSent = ""

    #removing stop words
    for token in doc:
        
        if(token.is_stop == True):
            continue
        else:
            inputSent = inputSent + " " + token.lemma_
        
        #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
    
    #lower case of the sentence
    inputSent = inputSent.lower()
    #remove numbers
    inputSent = re.sub('\d+', '', inputSent)
    #remover punctuation
    inputSent = inputSent.translate(str.maketrans('', '', string.punctuation))
    #remove multiple whitespaces
    inputSent = ' '.join(inputSent.split())
    
    
    
    #print(inputSent)
    return str(inputSent)

In [31]:
#computeCosineSimilarity("The cat chases the dog", "The dog chases the cat")

In [28]:
df = pd.read_csv('IBM_shuffled.csv', delimiter=",")

In [32]:
df = pd.read_csv('HR_IT_FM_shuffled.csv', delimiter = ";")

In [33]:
uniqueLabels = df.Label.unique()

foundLabels = []
classifiedSamplesInit = []

tfIdfList = []
allSamplesInit = []
allSamplesWithLabelInit = []

allSamplesMed = []
allSamplesWithLabelMed = []

allSamplesThird = []
allSamplesWithLabelThird = []

allSamplesOracled = []

In [34]:
#Save information from df in variables. Hereby, we compute the tf-idf values and eliminate stop words, numbers and more.

for ind in df.index:
    #preprocessedSentence = preprocessing(df['Text'][ind])
    preprocessedSentence = (df['Text'][ind])
    allSamplesWithLabelInit.append([preprocessedSentence, df['Label'][ind]])
    allSamplesInit.append(preprocessedSentence)

#Get TF-IDF Values
analyze = vectorizer.build_analyzer()
X = vectorizer.fit_transform(allSamplesInit)
(vectorizer.get_feature_names())
tfIdfList = X.toarray()

In [35]:
#Set up a list for further work with tuples of the format (text, label, tf-idf-values)
counter = 0
for sample in allSamplesWithLabelInit:
    text = sample[0]
    lable = sample[1]
    tfIdfVal = tfIdfList[counter]
    #embedding = nlp(text).vector 
    #embedding = getTfIdfSentenceEmbedding(text, counter)
    #embedding = getFasttextSentenceEmbedding(text, counter, False)
    embedding = model.encode(text)
    allSamplesWithLabelMed.append([text, lable, tfIdfVal, embedding])
    
    counter = counter + 1 
    
#The oracle assigns the label for one sample per category    

for x in allSamplesWithLabelMed:

    wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[3]]
    
    if(not(x[1] in foundLabels)):
        foundLabels.append(x[1])
        classifiedSamplesInit.append(wholeEntry)
    else: 
        allSamplesWithLabelThird.append(x)
        
    allSamplesOracled.append(wholeEntry)
 

sentenceDict = {}
similarityDict = {}

for label in uniqueLabels:
    sentenceDict.update({label: ""})
    
for label in uniqueLabels:
    similarityDict.update({label: 0})
    
#print(similarityDict, '\n \n')

In [36]:
len(allSamplesWithLabelThird)

599

In [37]:
len(classifiedSamplesInit)

3

In [38]:
def addOracledSamples(listOfSamples, samplesPerCategory):
    
    startDict = {}
    
    for label in uniqueLabels:
        startDict.update({label: 0})
        
    classifiedSamples1 = []
        
    limit = samplesPerCategory * len(uniqueLabels)
    
    unclassifiedList = listOfSamples[:]
    
    unclassifiedListFinal = []
    
    #print(limit, len(unclassifiedList))
    #print(startDict)
    
    for x in unclassifiedList:
        #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, embedding
        wholeEntry = [x[0], x[1], x[2], x[1], "oracle", x[5]]
        
        amountOfSamplesPerLabel = startDict.get(x[1])
        
        if(amountOfSamplesPerLabel < samplesPerCategory):
            #print(x[1], amountOfSamplesPerLabel)
            classifiedSamples1.append(wholeEntry)
            amountOfSamplesPerLabel = amountOfSamplesPerLabel + 1
            startDict.update({x[1]: amountOfSamplesPerLabel})
            
            
            
            unclassifiedList.remove(x)
            
            
        if(limit == len(classifiedSamples1)):
            break
            
    for x in unclassifiedList:
        text = x[0]
        lable = x[1]
        tfIdfVal = x[2]
        
        unclassifiedListFinal.append([text, lable, tfIdfVal, x[5]])
    
            
            
    print(len(classifiedSamples1), len(unclassifiedListFinal))
    
    #helper = [i[1] for i in classifiedSamples1]
    
    #print(helper)
    
    #print(startDict) 
    
    return classifiedSamples1, unclassifiedListFinal
            
    

In [39]:
classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)

3 197


In [40]:
def resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2):
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    consideredSamples = allSamplesOracled[startIndex1:endIndex1] + allSamplesOracled[startIndex2:endIndex2]
    
    print(len(consideredSamples))
    
    classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(consideredSamples, 2)

In [41]:
resetClassifiedSamples(100, 300, 0, 0)

200
6 194


In [42]:
allUnclassifiedSamples = allSamplesWithLabelThird[:]
allClassifiedSamples = classifiedSamplesInit[:]

In [43]:
#Hereby, if we have 4 categories, we save the biggest similarity per each category with the new sample 
#If the similarites are more or less the same (DEFINE Threshold), then 
def findAndClassifyEqualSimilaritySamples(similarityHP, gapHP):
    
    maxSimilarityPerCategories = []
    
    for label in uniqueLabels:
        maxSimilarityPerCategories.append((label, 0))
    #print(maxSimilarityPerCategories)
    
    
    for unclassifiedSample in unclassifiedSamples4GridSearch:
    
        mostSimilarSample = 0
        maxCosSim = 0
        maxCosSimLabel = ""
        
        for label in uniqueLabels:
            similarityDict.update({label: 0})
    
        for classifiedSample in classifiedSamples4GridSearch:
            #cosSim = cosineSimilarity(unclassifiedSample[2], classifiedSample[2])
            #print(len(unclassifiedSample), len(classifiedSample))
            cosSim = computeSBertSimilarity((unclassifiedSample[3]), (classifiedSample[5]))
            classifiedLabel = classifiedSample[1]
            
            if(maxCosSim < cosSim):
                maxCosSim = cosSim
                maxCosSimLabel = classifiedLabel
            
            if(cosSim > similarityDict[classifiedLabel]):
                similarityDict [classifiedLabel] = cosSim
                sentenceDict [classifiedLabel] = unclassifiedSample[0]
        
        gapPassed = False
        
        for i in similarityDict:
            if(i != maxCosSimLabel):
                gap = maxCosSim - similarityDict[i]
                if(gap < gapHP):
                    gapPassed = True
                    break
        
        if(maxCosSim > similarityHP and gapPassed):
            # ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted
            entry = [unclassifiedSample[0], unclassifiedSample[1], unclassifiedSample[2], unclassifiedSample[1], "oracle", unclassifiedSample[3]]
            #print("entry ", entry)
            unclassifiedSamples4GridSearch.remove(unclassifiedSample)
            classifiedSamples4GridSearch.append(entry)
            
        elif(maxCosSim > similarityHP):
            #STRUCTURE OF classifiedSamplesInit: 
            #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
            entry = [unclassifiedSample[0], maxCosSimLabel, unclassifiedSample[2], unclassifiedSample[1], "predicted", unclassifiedSample[3]]
            #print("entry ", entry)
            unclassifiedSamples4GridSearch.remove(unclassifiedSample)
            classifiedSamples4GridSearch.append(entry)
            
 
        #print(similarityDict, unclassifiedSample[1])
        #print()
    #print("FINISHED ", len(allUnclassifiedSamples), len(allClassifiedSamples))
    print()
    return len(unclassifiedSamples4GridSearch)

In [44]:
def gridSearchAnd3FoldValidation (minSimilarity, maxSimilarity, similarityStep, minGap, maxGap, gapStep, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    similarity = minSimilarity
    
    while(similarity <= maxSimilarity):
        similarityGrid.append(similarity)
        similarity = similarity + similarityStep
        
    g = minGap

    while(g <= maxGap):
        gapGrid.append(g)
        g = g + gapStep
        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
        
    for sim in similarityGrid:
    
        
        for gap in gapGrid:
            
            print("NOW NEW COMBINATION: ", sim, gap)
            
            numberOfPredicted = -1
            numberOfOracled = -1

            localAccuracy = -1
            globalAccuracy = -1
            localAutomationRate = -1
            globalAutomationRate = -1

            localHarmonicScore = -1
            globalHarmonicScore = -1

            unlabelledSamples = 1000
            
            currentSim = sim
            currentGap = gap
            
            simCounter = 0
            gapCounter = 0
            counter = 0
            
            resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            
            while (unlabelledSamples != 0 and counter < 35):
                
                
                #print("sim, gap ", currentSim, currentGap)
                unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)      
            
                numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
                numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
                
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                    currentSim = currentSim - 0.01
                    simCounter = simCounter + 1
                    
                '''
                if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                    #currentGap = currentGap - 0.005
                    gapCounter = gapCounter + 1
                '''
                
                numberOfOracled = numberOfNewOracled
                numberOfPredicted = numberOfNewPredicted

                if(unlabelledSamples == 0):
                    counter = counter + 10000
                    
                #print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
                #print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
                ##print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
                #print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))
                
                #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
                counter = counter + 1 
            
            #harmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
            localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
            localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)
            
            globalAutomationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            globalAccuracy  = getPercentageCorrectPredictions(classifiedSamples4GridSearch)
            globalHarmonicScore = getHarmonicScore(globalAccuracy, globalAutomationRate, 0.5)
            
            
            if(globalHarmonicScore > maxHarmonicScore):
                maxHarmonicScore = localHarmonicScore
                bestAccuracy = localAccuracy
                bestAutomationRate = localAutomationRate
                bestGap = gap
                bestSim = sim
            
                
            #print("")
            #print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            
            
            
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
            #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
            #print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
            #print("")
            
            print(globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap)
            
            tempResults = [globalAccuracy, globalAutomationRate, globalHarmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap]
            allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestGap, bestSim)
    return allResults, dfColumnName
            

In [45]:
def validation (similarityGapGrid, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch

        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
    
    
    
        
    for simGap in similarityGapGrid:
        
        numberOfPredicted = -1
        numberOfOracled = -1
        accuracy = -1
        automationRate = -1
        unlabelledSamples = 1000

        currentSim = simGap[0]
        currentGap = simGap[1]


        simCounter = 0
        gapCounter = 0
        counter = 0
        
        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)

        while (unlabelledSamples != 0 and counter < 100):

            #print("during process ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

            unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)


            numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
            numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
            automationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            accuracy = getPercentageCorrectPredictions(classifiedSamples4GridSearch)

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                currentSim = currentSim - 0.01
                simCounter = simCounter + 1
                #currentGap = currentGap - 1

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                currentGap = currentGap - 0.005
                gapCounter = gapCounter + 1

            numberOfOracled = numberOfNewOracled
            numberOfPredicted = numberOfNewPredicted

            #print("unlab samples ", unlabelledSamples)

            if(unlabelledSamples == 0):
                counter = counter + 10000

            #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
            counter = counter + 1 

        harmonicScore = getHarmonicScore(accuracy, automationRate, 0.5)

        localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
        localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
        localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)

        if(harmonicScore > maxHarmonicScore):
            maxHarmonicScore = harmonicScore
            bestAccuracy = accuracy
            bestAutomationRate = automationRate
            bestSim = simGap[0]
            bestGap = simGap[1]


        #print("")
        #print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
        #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
        #print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
        #print("")

        print(accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1])

        tempResults = [accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1]]
        allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestSim, bestGap)
    return allResults, dfColumnName
            

In [46]:
startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = gridSearchAnd3FoldValidation(minSimilarity = 0.50, maxSimilarity = 0.56, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.031, gapStep = 0.01, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
6 94
Gap Grid:  [0.01, 0.02, 0.03]
Similarity Grid:  [0.5, 0.55]


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
NOW NEW COMBINATION:  0.5 0.01
100
6 94































0.45 0.92 0.5012106537530268 0.45 0.92 0.5012106537530268 8 92 0.5 0.01
NOW NEW COMBINATION:  0.5 0.02
100
6 94































0.5 0.86 0.5456852791878173 0.5 0.86 0.5456852791878173 14 86 0.5 0.02
NOW NEW COMBINATION:  0.5 0.03
100
6 94































0.52 0.83 0.5619791666666667 0.52 0.83 0.5619791666666667 17 83 0.5 0.03
NOW NEW COMBINATION:  0.55 0.01
100
6 94



































0.49 0.89 0.538395061728395 0.46875 0.9270833333333334 0.52018391521197 7 89 0.55 0.01
NOW NEW COMBINATION:  0.55 0.02
100
6 94



































0.49 0.89 0.538395061728395 0.46875 0.9270833333333334 0.52018391521197 7 89 0.55 0.02
NOW NEW COMBINATION:  0.55 0.03


In [ ]:
startIndex1, endIndex1, startIndex2, endIndex2 = 0,200,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,300,0,0
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults2, tfIdfColumns2 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,200,300
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults3, tfIdfColumns3 = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.91, similarityStep = 0.05 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)


#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

200
8 192
Gap Grid:  [0.01, 0.015, 0.02, 0.025, 0.030000000000000002, 0.035, 0.04, 0.045, 0.049999999999999996]
Similarity Grid:  [0.1, 0.15000000000000002, 0.2, 0.25, 0.3, 0.35, 0.39999999999999997, 0.44999999999999996, 0.49999999999999994, 0.5499999999999999, 0.6, 0.65, 0.7000000000000001, 0.7500000000000001, 0.8000000000000002, 0.8500000000000002, 0.9000000000000002]


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
NOW NEW COMBINATION:  0.1 0.01
200
8 192








0.55 0.82 0.5887728459530026 0.55 0.82 0.5887728459530026 36 164 0.1 0.01
NOW NEW COMBINATION:  0.1 0.015
200
8 192








0.655 0.71 0.6653075822603719 0.655 0.71 0.6653075822603719 58 142 0.1 0.015
NOW NEW COMBINATION:  0.1 0.02
200
8 192








0.695 0.66 0.6877061469265368 0.695 0.66 0.6877061469265368 68 132 0.1 0.02
NOW NEW COMBINATION:  0.1 0.025
200
8 192








0.725 0.61 0.6986571879936808 0.725 0.61 0.698657187993680









0.655 0.71 0.6653075822603719 0.655 0.71 0.6653075822603719 58 142 0.39999999999999997 0.015
NOW NEW COMBINATION:  0.39999999999999997 0.02
200
8 192








0.695 0.66 0.6877061469265368 0.695 0.66 0.6877061469265368 68 132 0.39999999999999997 0.02
NOW NEW COMBINATION:  0.39999999999999997 0.025
200
8 192








0.725 0.61 0.6986571879936808 0.725 0.61 0.6986571879936808 78 122 0.39999999999999997 0.025
NOW NEW COMBINATION:  0.39999999999999997 0.030000000000000002
200
8 192








0.755 0.57 0.7089785831960461 0.755 0.57 0.7089785831960461 86 114 0.39999999999999997 0.030000000000000002
NOW NEW COMBINATION:  0.39999999999999997 0.035
200
8 192








0.785 0.515 0.7105008787346222 0.785 0.515 0.7105008787346222 97 103 0.39999999999999997 0.035
NOW NEW COMBINATION:  0.39999999999999997 0.04
200
8 192








0.815 0.45 0.7012428298279157 0.815 0.45 0.7012428298279157 110 90 0.39999999999999997 0.04
NOW NEW COMBINATION:  0.39999999999999997 0.045
200
8 192








0.83 0.425




































0.825 0.435 0.6995614035087719 0.8241206030150754 0.4371859296482412 0.7001805904522613 112 87 0.65 0.04
NOW NEW COMBINATION:  0.65 0.045
200
8 192



































0.855 0.4 0.6965376782077394 0.8542713567839196 0.4020100502512563 0.6973643728848322 119 80 0.65 0.045
NOW NEW COMBINATION:  0.65 0.049999999999999996
200
8 192



































0.86 0.385 0.6897916666666667 0.8592964824120602 0.3869346733668342 0.6906662750076059 122 77 0.65 0.049999999999999996
NOW NEW COMBINATION:  0.7000000000000001 0.01
200
8 192



































0.49 0.79 0.5302739726027397 0.46596858638743455 0.8272251308900523 0.510561973989006 33 158 0.7000000000000001 0.01
NOW NEW COMBINATION:  0.7000000000000001 0.015
200
8 192



































0.565 0.745 0.5936882933709449 0.5445026178010471 0.7801047120418848 0.5795063575168287 42 149 0.7000000000000001 0.015
NOW NEW COMBINATION:  0.7000000000000001 0.02



































0.705 0.43 0.6250515463917526 0.40404040404040403 0.8686868686868687 0.4524410774410775 13 86 0.9000000000000002 0.01
NOW NEW COMBINATION:  0.9000000000000002 0.015
200
8 192



































0.745 0.405 0.6378964059196617 0.48484848484848486 0.8181818181818182 0.5278592375366569 18 81 0.9000000000000002 0.015
NOW NEW COMBINATION:  0.9000000000000002 0.02
200
8 192



































0.775 0.38 0.64161220043573 0.5454545454545454 0.7676767676767676 0.5789740985271712 23 76 0.9000000000000002 0.02
NOW NEW COMBINATION:  0.9000000000000002 0.025
200
8 192



































0.83 0.33 0.6369767441860466 0.6565656565656566 0.6666666666666666 0.6585612968591692 33 66 0.9000000000000002 0.025
NOW NEW COMBINATION:  0.9000000000000002 0.030000000000000002
200
8 192



































0.85 0.295 0.6176108374384237 0.696969696969697 0.5959595959595959 0.6741182314952807 40 59 0.9000000000000002 0.0









0.75 0.5 0.6818181818181818 0.75 0.5 0.6818181818181818 100 100 0.3 0.04
NOW NEW COMBINATION:  0.3 0.045
200
8 192








0.8 0.42 0.6774193548387097 0.8 0.42 0.6774193548387097 116 84 0.3 0.045
NOW NEW COMBINATION:  0.3 0.049999999999999996
200
8 192








0.825 0.395 0.6774948024948023 0.825 0.395 0.6774948024948023 121 79 0.3 0.049999999999999996
NOW NEW COMBINATION:  0.35 0.01
200
8 192








0.505 0.82 0.5470277410832233 0.505 0.82 0.5470277410832233 36 164 0.35 0.01
NOW NEW COMBINATION:  0.35 0.015
200
8 192








0.53 0.785 0.5668256130790192 0.53 0.785 0.5668256130790192 43 157 0.35 0.015
NOW NEW COMBINATION:  0.35 0.02
200
8 192








0.58 0.715 0.6027616279069766 0.58 0.715 0.6027616279069766 57 143 0.35 0.02
NOW NEW COMBINATION:  0.35 0.025
200
8 192








0.62 0.645 0.62484375 0.62 0.645 0.62484375 71 129 0.35 0.025
NOW NEW COMBINATION:  0.35 0.030000000000000002
200
8 192








0.67 0.575 0.6485690235690237 0.67 0.575 0.6485690235690237 85 115 0.35 0.0



























0.75 0.49 0.6780442804428044 0.75 0.49 0.6780442804428044 102 98 0.6 0.035
NOW NEW COMBINATION:  0.6 0.04
200
8 192


























0.785 0.44 0.6785854616895876 0.785 0.44 0.6785854616895876 112 88 0.6 0.04
NOW NEW COMBINATION:  0.6 0.045
200
8 192


























0.88 0.365 0.6863247863247863 0.88 0.365 0.6863247863247863 127 73 0.6 0.045
NOW NEW COMBINATION:  0.6 0.049999999999999996
200
8 192


























0.885 0.33 0.6622448979591837 0.885 0.33 0.6622448979591837 134 66 0.6 0.049999999999999996
NOW NEW COMBINATION:  0.65 0.01
200
8 192



































0.465 0.825 0.5094621513944223 0.4595959595959596 0.8333333333333334 0.5048823790501553 33 165 0.65 0.01
NOW NEW COMBINATION:  0.65 0.015
200
8 192



































0.495 0.775 0.5335535465924895 0.4898989898989899 0.7828282828282829 0.5295281968922135 43 155 0.65 0.015
NOW NEW COMBINATION:  0.65 0.02
200
8 192






























In [ ]:
df1 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)
df2 = DataFrame (tfIdfResults2, columns= tfIdfColumns2)
df3 = DataFrame (tfIdfResults3, columns= tfIdfColumns3)
df1

In [ ]:
#df1.to_excel("TF-IDF-Benchmark.xlsx", sheet_name='1')  
#df2.to_excel("TF-IDF-Benchmark.xlsx", sheet_name='2')  
#df3.to_excel("TF-IDF-Benchmark.xlsx", sheet_name='3')

In [ ]:
with pd.ExcelWriter("Sentence-Bert-2_1-perCat-CFV.xlsx") as writer:
    df1.to_excel(writer, sheet_name='1', index=False)
    df2.to_excel(writer, sheet_name='2', index=False)
    df3.to_excel(writer, sheet_name='3', index=False)

In [141]:
resetClassifiedSamples(0,200,0,0)

200
4 196


In [158]:
similarityHP = 0.5
gapHP = 0.03
#resetClassifiedSamples(0,200,0,0)
findAndClassifyEqualSimilaritySamples(similarityHP, gapHP)

0

In [215]:
similarityGapGrid = [[0.5, 0.03], [0.55, 0.03], [0.5,0.01]]
startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0

In [216]:
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)

100
4 96


In [217]:
tfIdfResults3, tfIdfColumns3 = validation(similarityGapGrid, startIndex1, endIndex1, startIndex2, endIndex2)

Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96











100
4 96
0.52 0.75 0.5539772727272728 0.52 0.75 0.5539772727272728 25 75 0.5 0.03
100
4 96
















100
4 96
0.65 0.59 0.6370431893687708 0.65 0.59 0.6370431893687708 41 59 0.55 0.03
100
4 96











100
4 96
0.35 0.89 0.3983375959079284 0.35 0.89 0.3983375959079284 11 89 0.5 0.01
Best:  0.6370431893687708 0.65 0.59 0.55 0.03


In [159]:
print("Automation Rate ",getPercentageOfPredicted(classifiedSamples4GridSearch))
print("Accuracy ",getPercentageCorrectPredictions(classifiedSamples4GridSearch))
print("Number of Predicted ", getNumberOfPredicted(classifiedSamples4GridSearch))
print("Number of Oracled ",getNumberOfOracled(classifiedSamples4GridSearch))

Automation Rate  0.54
Accuracy  0.79
Number of Predicted  108
Number of Oracled  92


In [43]:

startIndex1, endIndex1, startIndex2, endIndex2 = 200,300,0,0
#to be changed
simGapGrid1 = [[0.5, 0.03],[0.7,0.035]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults11, tfIdfColumns11 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 0,100,0,0
#to be changed
simGapGrid2 = [[0.6, 0.04], [0.6,0.03], [0.6,0.035]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults22, tfIdfColumns22 = validation(simGapGrid2, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

startIndex1, endIndex1, startIndex2, endIndex2 = 100,200,0,0
#to be changed
simGapGrid3 = [[0.8,0.025],[0.5,0.035],[0.7,0.025],[0.7,0.03],[0.8,0.03],[0.1,0.04],[0.2,0.04],[0.3,0.04],[0.4,0.04],[0.5,0.03],[0.1,0.03],[0.2,0.03],[0.3,0.03],[0.4,0.03]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults33, tfIdfColumns33 = validation(simGapGrid3, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
#tfIdfResults, tfIdfColumns = gridSearchAnd3FoldValidation(minSimilarity = 0.10, maxSimilarity = 0.3, similarityStep = 0.06 , minGap = 0.01, maxGap = 0.05, gapStep = 0.005, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96











100
4 96
0.52 0.75 0.5539772727272728 0.52 0.75 0.5539772727272728 25 75 0.5 0.03
100
4 96



















































100
4 96
0.67 0.62 0.6593650793650795 0.67 0.62 0.6593650793650795 38 62 0.7 0.035
Best:  0.6593650793650795 0.67 0.62 0.7 0.035
100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96


























100
4 96
0.8 0.39 0.6610169491525424 0.8 0.39 0.6610169491525424 61 39 0.6 0.04
100
4 96


























100
4 96
0.73 0.52 0.6754448398576512 0.73 0.52 0.6754448398576512 48 52 0.6 0.03
100
4 96


























100
4 96
0.74 0.47 0.6637404580152672 0.74 0.47 0.6637404580152672

In [44]:
df11 = DataFrame (tfIdfResults11, columns= tfIdfColumns11)
df22 = DataFrame (tfIdfResults22, columns= tfIdfColumns22)
df33 = DataFrame (tfIdfResults33, columns= tfIdfColumns33)

In [45]:
with pd.ExcelWriter("Sentence-Bert-Validation.xlsx") as writer:
    df11.to_excel(writer, sheet_name='1', index=False)
    df22.to_excel(writer, sheet_name='2', index=False)
    df33.to_excel(writer, sheet_name='3', index=False)

In [49]:

startIndex1, endIndex1, startIndex2, endIndex2 = 300,400,0,0
#to be changed
simGapGrid1 = [[0.5,0.03],[0.5,0.035],[0.1,0.04],[0.2,0.04],[0.3,0.04],[0.4,0.04],[0.5,0.04],[0.6,0.04],[0.7,0.04]]
resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults111, tfIdfColumns111 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)


import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 5000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

100
4 96
Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
100
4 96







100
4 96
0.64 0.64 0.64 0.64 0.64 0.64 36 64 0.5 0.03
100
4 96







100
4 96
0.68 0.57 0.6547297297297298 0.68 0.57 0.6547297297297298 43 57 0.5 0.035
100
4 96







100
4 96
0.84 0.36 0.6631578947368422 0.84 0.36 0.6631578947368422 64 36 0.1 0.04
100
4 96







100
4 96
0.84 0.36 0.6631578947368422 0.84 0.36 0.6631578947368422 64 36 0.2 0.04
100
4 96







100
4 96
0.84 0.36 0.6631578947368422 0.84 0.36 0.6631578947368422 64 36 0.3 0.04
100
4 96







100
4 96
0.84 0.36 0.6631578947368422 0.84 0.36 0.6631578947368422 64 36 0.4 0.04
100
4 96







100
4 96
0.72 0.51 0.6652173913043479 0.72 0.51 0.6652173913043479 49 51 0.5 0.04
100
4 96



















100
4 96
0.73 0.51 0.6720216606498195 0.73 0.51 0.6720216606498195 49 51 0.6 0.04
100
4 96




































KeyboardInterrupt: 

In [50]:
df111 = DataFrame (tfIdfResults111, columns= tfIdfColumns111)

In [51]:
with pd.ExcelWriter("Sentence-Bert-Final.xlsx") as writer:
    df111.to_excel(writer, sheet_name='1', index=False)
    

In [516]:
getPercentageOfPredicted(allClassifiedSamples)

0.5253807106598984

In [517]:
getPercentageCorrectPredictions(allClassifiedSamples)

0.7690355329949239

In [518]:
getNumberOfPredicted(allClassifiedSamples)

207

In [519]:
getNumberOfOracled(allClassifiedSamples)

187

In [47]:
#Hereby, if we have 4 categories, we save the biggest similarity per each category with the new sample 
#If the similarites are more or less the same (DEFINE Threshold), then 
def findAndClassifyEqualSimilaritySamples(similarityHP, gapHP):
    
    maxSimilarityPerCategories = []
    
    for label in uniqueLabels:
        maxSimilarityPerCategories.append((label, 0))
    #print(maxSimilarityPerCategories)
    
    
    for unclassifiedSample in unclassifiedSamples4GridSearch:
    
        mostSimilarSample = 0
        maxCosSim = 0
        maxCosSimLabel = ""
        
        for label in uniqueLabels:
            similarityDict.update({label: 0})
    
        for classifiedSample in classifiedSamples4GridSearch:
            cosSim = cosineSimilarity(unclassifiedSample[2], classifiedSample[2])
            #print(len(unclassifiedSample), len(classifiedSample))
            #cosSim = computeSBertSimilarity((unclassifiedSample[3]), (classifiedSample[5]))
            classifiedLabel = classifiedSample[1]
            
            if(maxCosSim < cosSim):
                maxCosSim = cosSim
                maxCosSimLabel = classifiedLabel
            
            if(cosSim > similarityDict[classifiedLabel]):
                similarityDict [classifiedLabel] = cosSim
                sentenceDict [classifiedLabel] = unclassifiedSample[0]
        
        gapPassed = False
        
        for i in similarityDict:
            if(i != maxCosSimLabel):
                gap = maxCosSim - similarityDict[i]
                if(gap < gapHP):
                    gapPassed = True
                    break
        
        if(maxCosSim > similarityHP and gapPassed):
            # ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted
            entry = [unclassifiedSample[0], unclassifiedSample[1], unclassifiedSample[2], unclassifiedSample[1], "oracle", unclassifiedSample[3]]
            #print("entry ", entry)
        
            try:
                unclassifiedSamples4GridSearch.remove(unclassifiedSample)
                classifiedSamples4GridSearch.append(entry)
            except:
                print("entry ", entry)
        
        elif(maxCosSim > similarityHP):
            #STRUCTURE OF classifiedSamplesInit: 
            #ticketText, predictedLabel, TF-IDF, actualLabel, Oracle/Predicted, wordVector
            entry = [unclassifiedSample[0], maxCosSimLabel, unclassifiedSample[2], unclassifiedSample[1], "predicted", unclassifiedSample[3]]
            #print("entry ", entry)
            
            try:
                unclassifiedSamples4GridSearch.remove(unclassifiedSample)
                classifiedSamples4GridSearch.append(entry)
            except:
                print("entry ", entry)
            
            
            
 
        #print(similarityDict, unclassifiedSample[1])
        #print()
    #print("FINISHED ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
    #print()
    return len(unclassifiedSamples4GridSearch)

In [48]:
def validation (similarityGapGrid, startIndex1, endIndex1, startIndex2, endIndex2):
    
    similarityGrid = []
    gapGrid = []
    
    allResults = []
    dfColumnName = ["Accuracy", "Automation Rate", "Harmonic Score", "Local Accuracy", "Local Automation Rate", "Local Harmonic Score", "Number Of Oracled", "Number Of Predicted", "Minimal Similarity", "Minimal Gap"]
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch

        
    print("Gap Grid: ", gapGrid)
    print("Similarity Grid: ", similarityGrid)
    print("\n")
    #print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore," 
    #      "numberOfOracled, numberOfPredicted, sim, gap, currentSim, currentGap")
    print("accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap")
    
    
    maxHarmonicScore = 0
    bestGap = 0
    bestSim = 0
    bestAccuracy = 0
    bestAutomationRate = 0
    
    
    
        
    for simGap in similarityGapGrid:
        
        numberOfPredicted = -1
        numberOfOracled = -1
        accuracy = -1
        automationRate = -1
        unlabelledSamples = 1000

        currentSim = simGap[0]
        currentGap = simGap[1]


        simCounter = 0
        gapCounter = 0
        counter = 0

        while (unlabelledSamples != 0 and counter < 100):

            #print("during process ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

            unlabelledSamples = findAndClassifyEqualSimilaritySamples(currentSim, currentGap)


            numberOfNewPredicted = getNumberOfPredicted(classifiedSamples4GridSearch)
            numberOfNewOracled = getNumberOfOracled(classifiedSamples4GridSearch)
            automationRate = getPercentageOfPredicted(classifiedSamples4GridSearch)
            accuracy = getPercentageCorrectPredictions(classifiedSamples4GridSearch)

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted):
                currentSim = currentSim - 0.01
                simCounter = simCounter + 1
                #currentGap = currentGap - 1

            if (numberOfNewOracled == numberOfOracled and numberOfNewPredicted == numberOfPredicted and simCounter > 12):
                currentGap = currentGap - 0.005
                gapCounter = gapCounter + 1

            numberOfOracled = numberOfNewOracled
            numberOfPredicted = numberOfNewPredicted

            #print("unlab samples ", unlabelledSamples)

            if(unlabelledSamples == 0):
                break

            #accuracy, automationRate, numberOfOracled, numberOfPredicted, currentSim, currentGap
            counter = counter + 1 

        harmonicScore = getHarmonicScore(accuracy, automationRate, 0.5)

        localAccuracy = getLocalAccuracy(classifiedSamples4GridSearch)
        localAutomationRate = getLocalAutomationRate(classifiedSamples4GridSearch)
        localHarmonicScore = getHarmonicScore(localAccuracy, localAutomationRate, 0.5)

        if(harmonicScore > maxHarmonicScore):
            maxHarmonicScore = harmonicScore
            bestAccuracy = accuracy
            bestAutomationRate = automationRate
            bestSim = simGap[0]
            bestGap = simGap[1]


        #print("")
        print("before reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))

        resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
        #classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:200], 1)
        print("after reset ", len(unclassifiedSamples4GridSearch), len(classifiedSamples4GridSearch))
        #print("")

        print(accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1])

        tempResults = [accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, simGap[0], simGap[1]]
        allResults.append(tempResults)
            
    print("Best: ", maxHarmonicScore, bestAccuracy, bestAutomationRate, bestGap, bestSim)
    return allResults, dfColumnName
            

In [54]:
classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(allSamplesOracled[:], 5)

15 587


In [55]:
def resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2):
    
    global classifiedSamples4GridSearch, unclassifiedSamples4GridSearch
    
    
    consideredSamples = allSamplesOracled[startIndex1:endIndex1] + allSamplesOracled[startIndex2:endIndex2]
    
    print(len(consideredSamples))
    
    classifiedSamples4GridSearch, unclassifiedSamples4GridSearch = addOracledSamples(consideredSamples, 5)

In [56]:
#1 per Cat

startIndex1, endIndex1, startIndex2, endIndex2 = 0, 600,0,0
simGapGrid1 = [[0.1,0.04]]
#resetClassifiedSamples(startIndex1, endIndex1, startIndex2, endIndex2)
tfIdfResults1, tfIdfColumns1 = validation(simGapGrid1, startIndex1 = startIndex1, endIndex1 = endIndex1, startIndex2 = startIndex2, endIndex2 = endIndex2)

import winsound
frequency = 1500  # Set Frequency To 2500 Hertz
duration = 3000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

Gap Grid:  []
Similarity Grid:  []


accuracy, automationRate, harmonicScore, localAccuracy, localAutomationRate, localHarmonicScore, numberOfOracled, numberOfPredicted, sim, gap
entry  ['Hey I am new to this company and would need some assistance for completing the request documents of compensation for travel expenses. Can you support me please?', 'HR', array([0., 0., 0., ..., 0., 0., 0.]), 'HR', 'predicted', array([ 0.7740533 ,  1.817038  ,  0.05062284, ..., -0.20760918,
       -0.93145126, -0.8106793 ], dtype=float32)]
entry  ['Hey, my boss asked me to enter my career and development goals into the internal system. I have no clue what he is talking about. FYI: We will have our yearly employee talk next week, maybe this will help you', 'HR', array([0., 0., 0., ..., 0., 0., 0.]), 'HR', 'oracle', array([ 0.9660251 ,  0.60912615, -0.36725298, ..., -0.77611136,
       -2.7699635 ,  0.34103307], dtype=float32)]
before reset  0 602
600
15 585
after reset  585 15
0.8637873754152824 0.559800

In [57]:
d1111 = DataFrame (tfIdfResults1, columns= tfIdfColumns1)


In [58]:
with pd.ExcelWriter("S-BERT-OwnDataSet-h1-5-2.xlsx") as writer:
    d1111.to_excel(writer, sheet_name='1', index=False)